In [10]:
from tensorflow import keras
from keras import layers
import numpy as np
import matplotlib.pyplot as plt
from load_and_prepare_data_functions import load_and_subsample_series, split_training_data

In [11]:
def mse_function(answer, prediction):
    mse = ((answer - prediction)**2).mean(axis=0)
    for i in range(len(answer.shape)-1):
        mse = mse.mean(axis=0)
    return mse

In [12]:
# Variables relating to the data you want to load
system = 'Lorentz'
x_transformation_type = 0
number_of_data_points = 10000
length_of_subsequence = 20
number_timesteps_predict = 10
c = 50
repetitions = 5

In [13]:
name = f"{system}_{number_of_data_points}"
test_filename = f'data_dictionaries/data_testing_{name}.npy'

observations, predictions = load_and_subsample_series(number_of_data_points,
                                                        system,
                                                        length_of_subsequence + number_timesteps_predict,
                                                        number_timesteps_predict = number_timesteps_predict,
                                                        x_transformation_type = x_transformation_type,
                                                        c = c,
                                                        filename = test_filename)

# Split into training and test data
train_X, test_X, train_answer, test_answer = split_training_data(observations,
                                                                    number_timesteps_predict,
                                                                    predictions = predictions,
                                                                    frac=0.9)

In [16]:
finalmse = mse_function(test_X['input_ob'][:,-1,:], test_X['input_pred'][:,:])
print(f"mse of imperfect predictions = {finalmse}")
not_random_loss = np.zeros((repetitions))
random_loss = np.zeros((repetitions))

for i in range(repetitions):
    # Physical error ML model
    save_name = f"{system}{x_transformation_type}_c{str(c).replace('.','')}_{number_of_data_points}_{length_of_subsequence}_{number_timesteps_predict}"
    save_filepath = f'saved_models/random/physical_error_notrandom_number{i}_{save_name}'
    loaded_model = keras.models.load_model(save_filepath)
    loss_physical_error = loaded_model.evaluate(x = test_X, y = test_answer - test_X['input_pred'])
    print(f"loss physical error = {loss_physical_error}")
    not_random_loss[i] = loss_physical_error

    # Physical error random ML model
    save_filepath = f'saved_models/random/physical_error_random_number{i}_{save_name}'
    loaded_model = keras.models.load_model(save_filepath)
    loss_physical_error_random = loaded_model.evaluate(x = test_X, y = test_answer - test_X['input_pred'])
    print(f"loss physical error random = {loss_physical_error_random}")
    random_loss[i] = loss_physical_error_random

mse of imperfect predictions = 0.013959053877932339
32/32 [==============================] - 1s 6ms/step - loss: 0.0011
loss physical error = 0.0010754427639767528
32/32 [==============================] - 1s 6ms/step - loss: 0.0020
loss physical error random = 0.0019734068773686886
32/32 [==============================] - 1s 6ms/step - loss: 0.0021
loss physical error = 0.002084519015625119
32/32 [==============================] - 1s 11ms/step - loss: 0.0015
loss physical error random = 0.0015450201462954283
32/32 [==============================] - 2s 8ms/step - loss: 0.0020
loss physical error = 0.001998846186324954
32/32 [==============================] - 1s 8ms/step - loss: 0.0021
loss physical error random = 0.002055499469861388
32/32 [==============================] - 1s 12ms/step - loss: 0.0018
loss physical error = 0.0018332668114453554
32/32 [==============================] - 2s 8ms/step - loss: 0.0019
loss physical error random = 0.0019457114394754171
32/32 [==================

In [17]:
mean_loss_not_random = np.mean(not_random_loss)
mean_loss_random = np.mean(random_loss)
std_loss_not_random = np.std(not_random_loss)
std_loss_random = np.std(random_loss)

print(mean_loss_not_random, mean_loss_random, std_loss_not_random, std_loss_random)

0.0015857228892855347 0.0018204088555648923 0.00048219728064761156 0.00021300744843930793
